### Install packages

In [ ]:
try: # Install microgrids package in JupyterLite (if run in JupyterLite)
    import piplite
    await piplite.install([ 'ipywidgets', ])
except ImportError:
    pass

### Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import VBox, interactive_output, Output

### Define fixed variables

In [ ]:
# Constants
distance = 30.8  # Route Kilometer
trainWeight = 200  # Weight of Train in Tons
depots = 1 # number of additional depots or percentage of depot to consider
depotTrainPower = 1 # Power used by Trains in Depot
elevatedStations = 16 # number of elevated stations
atGradeStations = 4 # number of stations at grade
ugStations = 3 # number of underground stations
tractionPF = 0.95 # Traction Power Factor
tractionLosses = 5 # Traction Power losses in percentage
auxiliaryPF = 0.95 # Auxillary Power Factor
auxiliaryLosses = 5 # Auxillary Power losses in percentage
dailyHours = 20 # Daily Operational Hours
diversityFactor = 0.5 # Diversity Factor for auxiliary power

### Function for calculating output based on passed parameteric values

In [ ]:
def calculate_metrics(sec, regeneration, headway):
    """
    Calculate required power wattage, units, and maximum demand.
    """
    # Traction Power Requirement using SEC KW/1000 km for finding traction power
    trainPerHour = 60 / headway * 2 # Trains per hour in both direction
    peakTractionPower = distance * trainWeight * sec * trainPerHour / 1000 /1000 # Power in MW
    regTractionPower = peakTractionPower * (1 - regeneration/100) + depotTrainPower # Account Regeneration
    netTractionPower = regTractionPower / (1 - tractionLosses / 100) / tractionPF # include losses in MVA

    # Auxiliary Power requirements of Station and Depot buildings
    stationLoad = (elevatedStations + atGradeStations) * 300 + ugStations * 2500 # Auxiliary power for stations
    depotLoad = depots * 500 # Auxiliary power for Depot
    totalAuxiliaryPower = (stationLoad + depotLoad) / 1000 # Total of station and Depot in MW
    netAuxiliaryPower = (totalAuxiliaryPower / (1 - auxiliaryLosses / 100) / auxiliaryPF) # include losses in MVA
    
    watt = netTractionPower + netAuxiliaryPower

    tractionUnits = netTractionPower * dailyHours * 365 / 1000 # Yearly units consumption for Traction
    auxiliaryUnits = netAuxiliaryPower * diversityFactor * dailyHours * 365 / 1000 # Yearly units consumption for Auxiliary

    unit = tractionUnits + auxiliaryUnits
    
    #print(f'Traction: {netTractionPower:.2f} MW, Auxiliary: {netAuxiliaryPower:.2f} MW')
    #print(f'Traction: {tractionUnits:.2f} Mil, Auxiliary: {auxiliaryUnits:.2f} Mil')
    
    demand = (netTractionPower / 0.95 + netAuxiliaryPower / 0.95) / 0.95 # Assume 5% transimission losses
    
    return watt, unit, demand


### Plot the output

In [ ]:

def plot_metrics(sec, regeneration, headway):
    """
    Generate the plot for the number Power Consumption.
    """
    watt, unit, demand = calculate_metrics(sec, regeneration, headway)

    print(f'Total Power: {watt:.2f} MW, Units: {unit:.2f} Million, Maxm Demand: {int(demand)}')

    fig, ax = plt.subplots(figsize=(7, 5))  # Set figure size
    metrics = ['Total Power Required', 'Units Consumed', 'Maximum Demand']
    values = [watt, unit, demand]
    
    ax.bar(metrics, values, width=0.5)
    ax.set_title('Power Consumption Metrics')
    #ax.set_ylim(0, 55)
    ax.set_ylabel('Values')

    for i, v in enumerate(values):
        ax.text(i, v + 1, f"{v:.2f}", ha='center', va='bottom')

    return fig



### Update the paramateric value interactively to observe senstivity

In [ ]:
def update(sec, regeneration, headway):
    """
    Update the output plot based on the slider values.
    """
    out1.clear_output(wait=True)
    with out1:
        fig1 = plot_metrics(sec, regeneration, headway)
        plt.show(fig1)



### Widget and others

In [ ]:
# Widgets for interactive plots
sec_slider = widgets.FloatSlider(value=50, min=35, max=80, step=0.5, description='SEC')
regeneration_slider = widgets.FloatSlider(value=30, min=10, max=60, step=1, description='Regeneration %')
headway_slider = widgets.FloatSlider(value=4, min=1, max=15, step=0.1, description='Headway (min)')

# Output widget for plots
out1 = Output()

# Interactive display
ui = VBox([sec_slider, regeneration_slider, headway_slider])
interactive_plot = interactive_output(update, {'sec': sec_slider, 'regeneration': regeneration_slider, 'headway': headway_slider})

### Show interactive plot

In [ ]:
display(VBox([ui, out1, interactive_plot]))